In [32]:
# coding: utf-8
"""
igraphを使ったグラフクラスタリング
greedyアルゴリズム（louvain法）によってクラスタリングした後
サブグラフの中心性指標を図ることで単語の順位付けを行う
pagerankによるp(word|topic)の計算
purityによるスコアの計算
"""
from igraph import *
import csv
import collections
import pickle
import numpy as np

# csvファイルの読み込み
def readcsv(path):
    f = open(path, "rb")
    dataReader = csv.reader(f)
    arr = [row for row in dataReader]
    return arr

def writecsv(arr, path):
    f = open(path, "ab")
    dataWriter = csv.writer(f)
    dataWriter.writerows(arr)
    f.close()

def readdump(path):
    f = open(path, "r")
    arr = pickle.load(f)
    f.close()
    return arr

# 有向エッジリストを入力して、重み付き無向ネットワークを出力する
def cal_edgelist_to_network(list_edge):
    # 有向エッジリストを無向エッジリストに変換する
    list_edge = [tuple(sorted(row)) for row in list_edge]
    # ノードリスト
    list_vertices = list(set([word for row in list_edge for word in row]))
    # エッジリストとそのweightを作成
    tuple_edge, tuple_weight = zip(*collections.Counter(list_edge).items())
    return {"vertex": list_vertices, "edge": list(tuple_edge), "weight": list(tuple_weight)}

# クラスタリング済みのネットワークを元にサブグラフのリストを作成
# vertexには全てのvertexを代入する（PageRankを計算するため）
def cal_cluster_to_network(dict_network):
    if dict_network.has_key("cluster") == False:
        print "クラスタリングができていません"
    
    # クラスタごとにwordをまとめる
    dict_cluster = collections.defaultdict(list)
    for word, cluster in zip(dict_network["vertex"], dict_network["cluster"]):
        dict_cluster[cluster].append(word)

    # リストに変換
    list_cluster_vertex = [row[1] for row in dict_cluster.items()]
    
    # 同様にエッジとウェイトのリストも作成する
    list_cluster_edge = []
    list_cluster_weight = []
    for cluster_vertex in list_cluster_vertex:
        list_cluster_edge_one = []
        list_cluster_weight_one = []
        # エッジリストの中に、一つでもノードが含まれていれば、そのクラスのノードに含める
        for row, weight in zip(dict_network["edge"], dict_network["weight"]):
            # and と or を切り替えることによって性能の比較
            if row[0] in cluster_vertex or row[1] in cluster_vertex:
                list_cluster_edge_one.append(row)
                list_cluster_weight_one.append(weight)
        list_cluster_edge.append(list_cluster_edge_one)
        list_cluster_weight.append(list_cluster_weight_one)
    
    # まとめる
    list_dict_network = [{"vertex": dict_network["vertex"],
                          "edge": cluster_edge,
                          "weight": cluster_weight}
                         for cluster_edge, cluster_weight
                         in zip(list_cluster_edge, list_cluster_weight)]
    
    return list_dict_network

# f_measureを計算する
def cal_f_measure(list_predict_measure):
    # 生成したクラスタ内のカウント
    dict_predict_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_predict_cluster[row[0]].append(row[1])
        
    # もとあるクラス内のカウント
    dict_measure_cluster = collections.defaultdict(list)
    for row in list_predict_measure:
        dict_measure_cluster[row[1]].append(row[0])
    
    # local_purityの計算
    list_purity = []
    for row in dict_predict_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_purity.append([major_class, class_num])
    purity = float(np.sum(zip(*list_purity)[0])) / np.sum(zip(*list_purity)[1])
    print "Purity: ", purity
    
    # inverse_purityの計算
    list_inverse_purity = []
    for row in dict_measure_cluster.items():
        major_class = sorted(collections.Counter(row[1]).items(), key=lambda x: x[1], reverse=True)[0][1]
        class_num = len(row[1])
        list_inverse_purity.append([major_class, class_num])
    inverse_purity = float(np.sum(zip(*list_inverse_purity)[0])) / np.sum(zip(*list_inverse_purity)[1])
    print "Inverse Purity: ", inverse_purity
    
    print "F-value: ", 2 / (1 / purity + 1 / inverse_purity)

###メイン部分
1. 有向エッジリストから無向重み付きエッジリストを作成
2. louvain法によるクラスタリング
3. クラスタごとにpagerankを計算し、p(word|topic)とする
4. クラスタごとにp(word|topic)を出力する辞書を作成

In [33]:
# エッジリストの読み込み
list_edge = readcsv("./files/list_edgelist20160127.csv")
# 元のネットワークを作成する（無向）
dict_network_master = cal_edgelist_to_network(list_edge)
# g = Graph(directed=True)
g_master = Graph()
g_master.add_vertices(dict_network_master["vertex"])
g_master.add_edges(dict_network_master["edge"])
# louvain法によるクラスタリング、vertexと同じ長さのクラスタ番号が書かれたリストがreturn
dict_network_master["cluster"] = g_master.community_fastgreedy(weights=dict_network_master["weight"]).as_clustering().membership
# クラスタ結果をもとにサブグラフのリストを作成
list_dict_network_sub = cal_cluster_to_network(dict_network_master)
# サブクラスタごとに中心性の計算
for i, dict_network_sub in enumerate(list_dict_network_sub):
    g_sub = Graph()
    g_sub.add_vertices(dict_network_sub["vertex"])
    g_sub.add_edges(dict_network_sub["edge"])
    list_dict_network_sub[i]["center_bet"] = g_sub.betweenness(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["center_eigen"] = g_sub.eigenvector_centrality(directed=False, weights=dict_network_sub["weight"])
    list_dict_network_sub[i]["page_rank"] = g_sub.pagerank(directed=False, weights=dict_network_sub["weight"])
print "クラスタ数: ", len(list_dict_network_sub)

# トピックごとにwordを入力したらp(word|topic)が出るような辞書を作成
list_dict_prob = []
for i in range(len(list_dict_network_sub)):
    list_word_page = sorted(zip(list_dict_network_sub[i]["vertex"], list_dict_network_sub[i]["page_rank"]), key=lambda x: x[1], reverse=True)
    list_dict_prob.append({row[0]: row[1] for row in list_word_page})

クラスタ数:  7


In [34]:
num = 6
for row in sorted(list_dict_prob[num].items(), key=lambda x: x[1], reverse=True):
    print row[0], row[1]

不満 0.0329276310103
親族 0.0192424078804
夫婦 0.019217130646
同士 0.0175008176253
方途 0.0165808103572
お金 0.0158318548443
年金 0.0145578799629
体 0.0145537104274
不自由 0.012044690936
交流 0.0110878445493
準備 0.0103815564981
コミュニケーション 0.00973765737908
心 0.00888973437136
我が家 0.00873095975028
運動 0.0086920640216
仕事 0.00867623632624
外部 0.00866870345558
嫌 0.00865861189627
ライフ 0.00832970355442
叔母 0.00808783452064
うれしい 0.00696436304798
本人 0.00687913819504
流動食 0.00681376846739
範囲 0.00674297999255
ロング 0.00658512474254
身 0.00646297988241
プログラム 0.0063504358419
口 0.00609000564493
大切 0.00605490526307
個人的 0.00604126144249
検討 0.00596991416546
不明 0.00509222927031
親身 0.00501384747321
ナイト 0.00491431268191
人 0.00484278976039
顔 0.0047327061179
メリット 0.0046578609978
存在 0.00441701901989
思い 0.00441423157402
雰囲気 0.00439356976244
ＡＴＭ 0.00431047654168
気分転換 0.00425695060955
身体 0.00425274022926
頭 0.00419468576149
特定 0.00410817064788
住まい 0.00393719177299
自分 0.00380141017447
目的 0.00379184584032
カバー 0.00379029086712
重要 0.0036985515596

### 定量評価部分
1. 形態素解析済みセンテンスを読み込み、確率を計算し、どのクラスに属するか計算
2. 予測クラスと実際クラスのリストを作成する

In [35]:
list_sep_words = readdump("./files/list_sep_words_label.dump")
# 確率が最大になるクラスを予想する
# 実質ラベルがintじゃない場合は、エラーとして、記録しない
list_predict_class = []
error_count = 0
for row in list_sep_words:
    try:
        class_tmp = 0
        prob_tmp = 0
        for num, dict_prob in enumerate(list_dict_prob):
            prob_tmp_tmp = 1
            for word in row[0]:
                prob_tmp_tmp *= float(dict_prob[word])
            if prob_tmp_tmp > prob_tmp:
                prob_tmp = prob_tmp_tmp
                class_tmp = num
        list_predict_class.append([class_tmp, int(row[1])])
    except:
        error_count += 1
print "エラーデータ数: ", error_count

エラーデータ数:  75


In [36]:
list_predict, list_class = zip(*list_predict_class)
collections.Counter(list(list_predict))

Counter({2: 770, 0: 709, 1: 595, 3: 483, 5: 474, 6: 151, 4: 104})

###計算結果の表示

In [37]:
cal_f_measure(list_predict_class)

Purity:  0.488131466829
Inverse Purity:  0.611990261716
F-value:  0.543088453551
